In [1]:
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
import sys

rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('font', size=20)
rc('text', usetex=True)
rc('xtick', labelsize=20, color='black')
rc('ytick', labelsize=20, color='black')
rc('lines', linewidth=3.0)
rcParams['axes.linewidth'] = 1.0
rcParams['axes.edgecolor'] = 'black'
rcParams['figure.figsize'] = (10.0, 8.0)
rc('text', usetex=True)



def linear_f (x,a,b):
    return a*x+b
def quadratic_f(x,a,b,c):
    return a*x**2+b*x+c
def cubic_f(x,a,b,c,d):
    return a*x**3+b*x*2+c*x+d
def quartic_f(x,a,b,c,d,e):
    return a*x**4+b*x**3+c*x**2+d*x+e
def quintic_f(x,a,b,c,d,e,f):
    return a*x**5+b*x**4+c*x**3+d*x**2+e*x+f
def exponential_f(x,a,b,c,d):
    return a+b*exp(c*x**d)
def powerlaw_f(x,a,b,c):
    return a+b*x**c

methoddic = {'linear': linear_f, 'quadratic': quadratic_f, 'cubic': cubic_f, 'quartic': quartic_f ,'quintic': quintic_f, 'exponential': exponential_f,'powerlaw': powerlaw_f}


def T_profile(elev_min,elev_max, fname):
    data = loadtxt(fname)
    elev_min = float(elev_min); elev_max = float(elev_max)
    if elev_min<min(-data[:,1]) or elev_max>max(-data[:,1]):
        sys.exit("elev_min or elev_max: not in the range!")
    else:
        elev = linspace(elev_min,elev_max,50)
        f_value = interp1d(-data[:,1],data[:,0],'linear')
        temp = f_value(elev)
        return (temp,elev)

def curvefit_temp(elev_min,elev_max,method, fname):
    """
    Inputs:
    elev_min: a real number between -4200.< elev_min < 0.
    elev_max: a real number between -4200.< elev_max < 0 & elev_max > elev_min
    method: choose one of the followings:
    - 'linear'   : y = f(x) = ax + b
    - 'quadratic': y = f(x) = ax^2 + b
    - 'cubic'    : y = f(x) = ax^3 + bx^2 + c*x + d
    - 'quartic'  : y = f(x) = ax^4 + bx^3 + c*x^2 + d*x + e
    - 'quintic'  : y = f(x) = ax^5 + bx^4 + c*x^3 + d*x^2 + ex + f
    fname: name of the input data file e.g. 'temperature.dat' 
    
    Example:   out = curvefit_temp(-3000.,-2000.,'linear', 'temperature.dat')
    
    Outputs: 
    - out[0] : temperature
    - out[1] : elevation
    - out[2] : optimized coefficients
    
    Note (for the example above): 
    -  out[2][0]: the "a" coefficient of the linear function
    -  out[2][1]: the "b" coefficient of the linear function
    """
    error = 1
    for key in methoddic:
        if method == key:
            error = 0
    if error == 1:
        sys.exit("wrong interpolation method: check your spelling!")
    xy = T_profile(elev_min,elev_max, fname)
    popt, pcov = curve_fit(methoddic[method],xy[1],xy[0])
    return (methoddic[method](xy[1],*popt),xy[1], popt)
